## Time Series Analysis on Sale- Regression Model

# Business understanding 

# Hypothesis
### Null Hypothesis: Promotions have no significant effect on product sales.
### Alternate Hypothesis:  Promotions positively impact product sales, leading to increased sales during promotional periods.


# 5 Analytic Question
 1)	What is the effect of seasonal variations (e.g., Monthly, or quarterly patterns) on total sales?
 2)	What store is the highest performer in terms of sales?
 3)	What are the best-selling products per location?
 4)	Does store size affect total sales?
 5)	Which days have the most sales?




### Python Libraries
### This section is to import all necessary libraries that will be used in the notebook

In [156]:
%pip install pyodbc  
%pip install python-dotenv 
%pip install openpyxl
#pip install --upgrade nbformat
%pip install plotly

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: pyodbc in c:\users\benedicta mankata\appdata\local\programs\python\python311\lib\site-packages (4.0.39)




[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


#### - Import all the necessary packages

In [157]:
import pyodbc #just installed with pip
from dotenv import dotenv_values #import the dotenv_values function from the dotenv package
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import plotly.express as px
import seaborn as sns
import warnings 

warnings.filterwarnings('ignore')

## 3. Data preparation
### 3.1: Data Loading
### This is how the first dataset was retrieved from the SQL Server database. The second and third datasets were downloaded and imported.

In [158]:
environment_variables = dotenv_values('.env')
    
database = environment_variables.get("DATABASE")
server = environment_variables.get("SERVER")
username = environment_variables.get("USERNAME")
password = environment_variables.get("PASSWORD")
    
connection_string = f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}"
    
connection = pyodbc.connect(connection_string)

# Data for the Project
1. First dataset - the oil,holiday and stores data was extracted from Microsoft SQL Server
2. Second dataset - the train and transaction of the dataset was downloaded from a GitHub repository
3. Third dataset - The test and submission of the data was downloaded from Onedrive

In [159]:
# extract data from Table 1: dbo.oil
# assign it to data_oil
# preview data_oil

query="Select * from dbo.oil" 
data_oil=pd.read_sql(query, connection)

data_oil.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.139999
2,2013-01-03,92.970001
3,2013-01-04,93.120003
4,2013-01-07,93.199997


In [160]:
data_oil.shape

(1218, 2)

In [161]:
data_oil.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        1218 non-null   object 
 1   dcoilwtico  1175 non-null   float64
dtypes: float64(1), object(1)
memory usage: 19.2+ KB


In [162]:
data_oil.isnull().sum()

date           0
dcoilwtico    43
dtype: int64

In [163]:
# Save extracted data as data_oil.csv
data_oil['date'] = pd.to_datetime(data_oil['date'])  # Convert 'date' column to datetime if not already
data_oil.to_csv('data_oil.csv', index=False)  # Save the DataFrame to CSV without the index column

In [164]:
# rename column 'dcoilwtico' to oil_price in oil.csv

data_oil.rename(columns={'dcoilwtico':'oil_prices'}, inplace=True)
data_oil.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        1218 non-null   datetime64[ns]
 1   oil_prices  1175 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 19.2 KB


In [165]:
# extract data from Table 2: dbo.holidays_events
# assign it to data_oil
# preview data_oil

query2="Select * from dbo.holidays_events" 
data_holiday=pd.read_sql(query2, connection)

data_holiday.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [166]:
data_holiday.shape

(350, 6)

In [167]:
data_holiday.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         350 non-null    object
 1   type         350 non-null    object
 2   locale       350 non-null    object
 3   locale_name  350 non-null    object
 4   description  350 non-null    object
 5   transferred  350 non-null    bool  
dtypes: bool(1), object(5)
memory usage: 14.1+ KB


In [168]:
data_holiday.isnull().sum()

date           0
type           0
locale         0
locale_name    0
description    0
transferred    0
dtype: int64

In [169]:
# Save extracted data as data_holiday.csv
data_holiday['date'] = pd.to_datetime(data_holiday['date'])  # Convert 'date' column to datetime if not already
data_holiday.to_csv('data_holiday.csv', index=False)  # Save the DataFrame to CSV without the index column



In [170]:
# extract data from Table 3: dbo.stores
# assign it to data_stores
# preview data_stores

query3="Select * from dbo.stores" 
data_stores=pd.read_sql(query3, connection)

data_stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [171]:
data_stores.shape

(54, 5)

In [172]:
data_stores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   store_nbr  54 non-null     int64 
 1   city       54 non-null     object
 2   state      54 non-null     object
 3   type       54 non-null     object
 4   cluster    54 non-null     int64 
dtypes: int64(2), object(3)
memory usage: 2.2+ KB


In [173]:
data_stores.isnull().sum()

store_nbr    0
city         0
state        0
type         0
cluster      0
dtype: int64

In [174]:
# Save extracted data as data3.csv

data_stores.to_csv('data_stores.csv')

In [175]:
data_sub = pd.read_excel('E:/LP3 dataset/sample_submission.xlsx')

In [176]:
data_sub.head(5)

,id,sales
0,3000888,0
1,3000889,0
2,3000890,0
3,3000891,0
4,3000892,0


In [177]:
data_trans = pd.read_csv('E:/LP3 dataset/transactions.csv', parse_dates=['date'])

In [178]:
data_trans.head(5)

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [179]:
data_trans.shape

(83488, 3)

In [180]:
data_trans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83488 entries, 0 to 83487
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          83488 non-null  datetime64[ns]
 1   store_nbr     83488 non-null  int64         
 2   transactions  83488 non-null  int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 1.9 MB


In [181]:
data_trans.isnull().sum()

date            0
store_nbr       0
transactions    0
dtype: int64

In [182]:
data_test= pd.read_excel('E:/LP3 dataset/test.xlsx', parse_dates=['date'])

In [183]:
data_test.head(5)

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


In [184]:
data_test.shape

(28512, 5)

In [185]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28512 entries, 0 to 28511
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id           28512 non-null  int64         
 1   date         28512 non-null  datetime64[ns]
 2   store_nbr    28512 non-null  int64         
 3   family       28512 non-null  object        
 4   onpromotion  28512 non-null  int64         
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 1.1+ MB


In [186]:
data_test.isnull().sum()

id             0
date           0
store_nbr      0
family         0
onpromotion    0
dtype: int64

In [187]:
data_train= pd.read_csv('E:/LP3 dataset/train.csv', parse_dates=['date'])

In [188]:
data_train.head(5)

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [189]:
data_train.shape

(3000888, 6)

In [190]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 6 columns):
 #   Column       Dtype         
---  ------       -----         
 0   id           int64         
 1   date         datetime64[ns]
 2   store_nbr    int64         
 3   family       object        
 4   sales        float64       
 5   onpromotion  int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(1)
memory usage: 137.4+ MB


In [191]:
data_train.isnull().sum()

id             0
date           0
store_nbr      0
family         0
sales          0
onpromotion    0
dtype: int64

### Checking the range of date 

In [192]:
# cheacking the range of date on data_oil
data_oil['date'].min(),data_oil['date'].max()

(Timestamp('2013-01-01 00:00:00'), Timestamp('2017-08-31 00:00:00'))

In [193]:
# cheacking the range of date on holiday
data_holiday['date'].min(),data_holiday['date'].max()

(Timestamp('2012-03-02 00:00:00'), Timestamp('2017-12-26 00:00:00'))

In [194]:
# cheacking the range of date on transaction 
data_trans['date'].min(),data_trans['date'].max()

(Timestamp('2013-01-01 00:00:00'), Timestamp('2017-08-15 00:00:00'))

In [195]:
# cheacking the range of date on train  
data_train['date'].min(),data_train['date'].max()

(Timestamp('2013-01-01 00:00:00'), Timestamp('2017-08-15 00:00:00'))

In [196]:
# cheacking the range of date on test 
data_test['date'].min(),data_test['date'].max()

(Timestamp('2017-08-16 00:00:00'), Timestamp('2017-08-31 00:00:00'))

### splitting the data_oil date 

In [197]:
# split oil.csv on the dates that reflects the split for the test and train
#define the start and end dates for the oil.csv that will be joined to the test

data_oil_test_start_date= '2017-08-16'
data_oil_test_end_date= '2017-08-31'

data_oil_train = data_oil[data_oil['date'] < data_oil_test_start_date]
data_oil_test = data_oil[data_oil['date'].between (data_oil_test_start_date, data_oil_test_end_date)]

# Verify the split
range_oil_test = data_oil_test['date'].min(), data_oil_test['date'].max()
range_oil_train = data_oil_train['date'].min(), data_oil_train['date'].max()

print(range_oil_test)
print(range_oil_train)


(Timestamp('2017-08-16 00:00:00'), Timestamp('2017-08-31 00:00:00'))
(Timestamp('2013-01-01 00:00:00'), Timestamp('2017-08-15 00:00:00'))


### Merging the dataset 

In [198]:
# merge test and oil_test on 'date'

merged_test = pd.merge( data_test, data_oil_test, on='date', how='left')
print(merged_test.head())

        id       date  store_nbr      family  onpromotion  oil_prices
0  3000888 2017-08-16          1  AUTOMOTIVE            0   46.799999
1  3000889 2017-08-16          1   BABY CARE            0   46.799999
2  3000890 2017-08-16          1      BEAUTY            2   46.799999
3  3000891 2017-08-16          1   BEVERAGES           20   46.799999
4  3000892 2017-08-16          1       BOOKS            0   46.799999


In [199]:
#check for null or missing values

merged_test.isnull().sum()

id                0
date              0
store_nbr         0
family            0
onpromotion       0
oil_prices     7128
dtype: int64

In [200]:
merged_test.shape

(28512, 6)

In [201]:
# merge train and data_oil_train_interpolated datasets
# print merged dataset to see if successfull

merged_train = pd.merge(data_train, data_oil_train, on='date', how='left')
print(merged_train.head())

   id       date  store_nbr      family  sales  onpromotion  oil_prices
0   0 2013-01-01          1  AUTOMOTIVE    0.0            0         NaN
1   1 2013-01-01          1   BABY CARE    0.0            0         NaN
2   2 2013-01-01          1      BEAUTY    0.0            0         NaN
3   3 2013-01-01          1   BEVERAGES    0.0            0         NaN
4   4 2013-01-01          1       BOOKS    0.0            0         NaN


In [202]:
merged_train.isnull().sum()

id                  0
date                0
store_nbr           0
family              0
sales               0
onpromotion         0
oil_prices     928422
dtype: int64

In [203]:
merged_train.shape

(3000888, 7)

In [204]:
# join transactions and stores on 'store_nbr'
transactions_store = pd.merge(data_trans, data_stores, on='store_nbr', how='left')
print(transactions_store.head())


        date  store_nbr  transactions     city        state type  cluster
0 2013-01-01         25           770  Salinas  Santa Elena    D        1
1 2013-01-02          1          2111    Quito    Pichincha    D       13
2 2013-01-02          2          2358    Quito    Pichincha    D       13
3 2013-01-02          3          3487    Quito    Pichincha    D        8
4 2013-01-02          4          1922    Quito    Pichincha    D        9


In [205]:
#checking the null value of the transactions_store
transactions_store.isnull().sum()

date            0
store_nbr       0
transactions    0
city            0
state           0
type            0
cluster         0
dtype: int64

In [206]:
# cheacking the range of date on the  transaction_store 
transactions_store['date'].min(),transactions_store['date'].max()

(Timestamp('2013-01-01 00:00:00'), Timestamp('2017-08-15 00:00:00'))

splitting the transaction_store date

In [207]:
#splitting the transactions_store date
# Define start and end dates for train and test sets
transactions_store_start_date = pd.Timestamp('2013-01-01')
transactions_store_end_date = pd.Timestamp('2017-08-15')

# Filter the DataFrame to get training and testing data
transactions_store_train = transactions_store[
    (transactions_store['date'] >= transactions_store_start_date) &
    (transactions_store['date'] <= transactions_store_end_date)
]

transactions_store_test = transactions_store[
    transactions_store['date'] > transactions_store_end_date
]

# Verify the split
transactions_store_train_min_date = transactions_store_train['date'].min()
transactions_store_train_max_date = transactions_store_train['date'].max()
transactions_store_test_min_date = transactions_store_test['date'].min()
transactions_store_test_max_date = transactions_store_test['date'].max()

print("Train Set Date Range:", transactions_store_train_min_date, "to", transactions_store_train_max_date)
print("Test Set Date Range:", transactions_store_test_min_date, "to", transactions_store_test_max_date)

Train Set Date Range: 2013-01-01 00:00:00 to 2017-08-15 00:00:00
Test Set Date Range: NaT to NaT


In pandas, NaT stands for "Not a Timestamp" and represents missing or undefined values in datetime-related data. Since the transactions_store dataset has the same timestamp as the train we decided to maintain the timestamp for the train and use NaT for the test data

Merging the transactions_store to the train and test data 

In [208]:
# merge test and transactions_store on 'date' on the test data

test_tran= pd.merge( merged_test, transactions_store_test, on=['date', 'store_nbr'], how='left')
print(test_tran.head())

        id       date  store_nbr      family  onpromotion  oil_prices  \
0  3000888 2017-08-16          1  AUTOMOTIVE            0   46.799999   
1  3000889 2017-08-16          1   BABY CARE            0   46.799999   
2  3000890 2017-08-16          1      BEAUTY            2   46.799999   
3  3000891 2017-08-16          1   BEVERAGES           20   46.799999   
4  3000892 2017-08-16          1       BOOKS            0   46.799999   

   transactions city state type  cluster  
0           NaN  NaN   NaN  NaN      NaN  
1           NaN  NaN   NaN  NaN      NaN  
2           NaN  NaN   NaN  NaN      NaN  
3           NaN  NaN   NaN  NaN      NaN  
4           NaN  NaN   NaN  NaN      NaN  


In [209]:
test_tran.isnull().sum()

id                  0
date                0
store_nbr           0
family              0
onpromotion         0
oil_prices       7128
transactions    28512
city            28512
state           28512
type            28512
cluster         28512
dtype: int64

In [210]:
test_tran.shape

(28512, 11)

In [211]:
# merge test and transactions_store on 'date' on the train data

train_tran= pd.merge( merged_train, transactions_store_train, on=['date','store_nbr'], how='left')
print(train_tran.head())

   id       date  store_nbr      family  sales  onpromotion  oil_prices  \
0   0 2013-01-01          1  AUTOMOTIVE    0.0            0         NaN   
1   1 2013-01-01          1   BABY CARE    0.0            0         NaN   
2   2 2013-01-01          1      BEAUTY    0.0            0         NaN   
3   3 2013-01-01          1   BEVERAGES    0.0            0         NaN   
4   4 2013-01-01          1       BOOKS    0.0            0         NaN   

   transactions city state type  cluster  
0           NaN  NaN   NaN  NaN      NaN  
1           NaN  NaN   NaN  NaN      NaN  
2           NaN  NaN   NaN  NaN      NaN  
3           NaN  NaN   NaN  NaN      NaN  
4           NaN  NaN   NaN  NaN      NaN  


In [212]:
train_tran.isnull().sum()

id                   0
date                 0
store_nbr            0
family               0
sales                0
onpromotion          0
oil_prices      928422
transactions    245784
city            245784
state           245784
type            245784
cluster         245784
dtype: int64

In [213]:
train_tran.shape

(3000888, 12)

In [214]:
train_tran.head(2)

,id,date,store_nbr,family,sales,onpromotion,oil_prices,transactions,city,state,type,cluster
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2013-01-01,1,BABY CARE,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [215]:
# cheacking the range of date on the  train_tran
train_tran['date'].min(),train_tran['date'].max()

(Timestamp('2013-01-01 00:00:00'), Timestamp('2017-08-15 00:00:00'))

In [216]:
# cheacking the range of date on the test_tran 
test_tran['date'].min(),test_tran['date'].max()

(Timestamp('2017-08-16 00:00:00'), Timestamp('2017-08-31 00:00:00'))

dropping some of the column at the holiday data 

In [233]:
# Drop columns 'locale' and 'locale_name'
holiday = data_holiday.drop(['locale', 'locale_name', 'description'], axis=1)

# Display the updated DataFrame
print(holiday.head())

        date     type  transferred
0 2012-03-02  Holiday        False
1 2012-04-01  Holiday        False
2 2012-04-12  Holiday        False
3 2012-04-14  Holiday        False
4 2012-04-21  Holiday        False


In [236]:
# Check for duplicate rows based on the 'date' column
holiday = dholiday_data.duplicated(subset=['date'])

# Display the DataFrame with duplicate rows
print(df[duplicate_rows])

0      False
1      False
2      False
3      False
4      False
       ...  
345    False
346    False
347    False
348    False
349    False
Length: 350, dtype: bool


In [218]:
# cheacking the range of date on the test_tran 
holiday['date'].min(),holiday['date'].max()

(Timestamp('2012-03-02 00:00:00'), Timestamp('2017-12-26 00:00:00'))

In [219]:
# Filter holiday data for the train and test periods
train_holiday = holiday[(holiday['date'] >= '2013-01-01') & (holiday['date'] <= '2017-08-15')]
test_holiday = holiday[(holiday['date'] >= '2017-08-16') & (holiday['date'] <= '2017-08-31')]

# Now we can merge the train and test holiday data with the respective train and test datasets
train = train_tran.merge(train_holiday, on=['date','type'], how='left')
test = test_tran.merge(test_holiday, on=['date','type'], how='left')

In [220]:
# cheacking the range of date on the train
train['date'].min(),train['date'].max()

(Timestamp('2013-01-01 00:00:00'), Timestamp('2017-08-15 00:00:00'))

In [221]:
train.head(2)

,id,date,store_nbr,family,sales,onpromotion,oil_prices,transactions,city,state,type,cluster,transferred
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2013-01-01,1,BABY CARE,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [222]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 13 columns):
 #   Column        Dtype         
---  ------        -----         
 0   id            int64         
 1   date          datetime64[ns]
 2   store_nbr     int64         
 3   family        object        
 4   sales         float64       
 5   onpromotion   int64         
 6   oil_prices    float64       
 7   transactions  float64       
 8   city          object        
 9   state         object        
 10  type          object        
 11  cluster       float64       
 12  transferred   object        
dtypes: datetime64[ns](1), float64(4), int64(3), object(5)
memory usage: 297.6+ MB


In [223]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28512 entries, 0 to 28511
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id            28512 non-null  int64         
 1   date          28512 non-null  datetime64[ns]
 2   store_nbr     28512 non-null  int64         
 3   family        28512 non-null  object        
 4   onpromotion   28512 non-null  int64         
 5   oil_prices    21384 non-null  float64       
 6   transactions  0 non-null      float64       
 7   city          0 non-null      object        
 8   state         0 non-null      object        
 9   type          0 non-null      object        
 10  cluster       0 non-null      float64       
 11  transferred   0 non-null      object        
dtypes: datetime64[ns](1), float64(3), int64(3), object(5)
memory usage: 2.6+ MB


## imputing the missing values in the train and test data 

In [224]:
train.isnull().sum()

id                    0
date                  0
store_nbr             0
family                0
sales                 0
onpromotion           0
oil_prices       928422
transactions     245784
city             245784
state            245784
type             245784
cluster          245784
transferred     3000888
dtype: int64

In [225]:
#imputing the missing value 
train['oil_prices'].fillna(method='bfill', inplace=True)
train['transactions'].fillna(method='bfill', inplace=True)
train['city'].fillna(method='bfill', inplace=True)
train['state'].fillna(method='bfill', inplace=True)
train['type'].fillna(method='bfill', inplace=True)
train['cluster'].fillna(method='bfill', inplace=True)
train['transferred'].fillna(method='bfill', inplace=True)

# Print the updated DataFrame
print(train.head())

   id       date  store_nbr      family  sales  onpromotion  oil_prices  \
0   0 2013-01-01          1  AUTOMOTIVE    0.0            0   93.139999   
1   1 2013-01-01          1   BABY CARE    0.0            0   93.139999   
2   2 2013-01-01          1      BEAUTY    0.0            0   93.139999   
3   3 2013-01-01          1   BEVERAGES    0.0            0   93.139999   
4   4 2013-01-01          1       BOOKS    0.0            0   93.139999   

   transactions     city        state type  cluster  transferred  
0         770.0  Salinas  Santa Elena    D      1.0          NaN  
1         770.0  Salinas  Santa Elena    D      1.0          NaN  
2         770.0  Salinas  Santa Elena    D      1.0          NaN  
3         770.0  Salinas  Santa Elena    D      1.0          NaN  
4         770.0  Salinas  Santa Elena    D      1.0          NaN  


In [226]:
train.isnull().sum()

id                    0
date                  0
store_nbr             0
family                0
sales                 0
onpromotion           0
oil_prices            0
transactions          0
city                  0
state                 0
type                  0
cluster               0
transferred     3000888
dtype: int64

after using the backward fill we still had missing values so we coming to use the summary Statistics to understand the missing values 

In [227]:
# Summary Statistics: Calculate percentage of missing values for each column
missing_percentage = (train.isnull().sum() / len(train)) * 100
missing_info = pd.DataFrame({'Column': train.columns, 'Missing Percentage': missing_percentage})
missing_info = missing_info.sort_values(by='Missing Percentage', ascending=False)
print(missing_info)

                    Column  Missing Percentage
transferred    transferred               100.0
id                      id                 0.0
date                  date                 0.0
store_nbr        store_nbr                 0.0
family              family                 0.0
sales                sales                 0.0
onpromotion    onpromotion                 0.0
oil_prices      oil_prices                 0.0
transactions  transactions                 0.0
city                  city                 0.0
state                state                 0.0
type                  type                 0.0
cluster            cluster                 0.0


In [228]:
test.isnull().sum()

id                  0
date                0
store_nbr           0
family              0
onpromotion         0
oil_prices       7128
transactions    28512
city            28512
state           28512
type            28512
cluster         28512
transferred     28512
dtype: int64

## EDA

# Plotting the time series data to visualize trends, seasonality, and patterns